In [ ]:
# Notebook 01: Synthetic Data Factory (Ollama Local)
import ollama
import json
import os

# 1. cargar frases de "Alicia en el País de las Maravillas" en chunks_nltk.txt
with open("../data/chunks_nltk.txt", "r", encoding="utf-8") as f:
    texto_alicia = f.read().split("\n\n")

print(f"Total de frases cargadas: {len(texto_alicia)}")

In [ ]:
# Función para generar el par sintético
def generar_par_sintetico(frase_original):
    prompt = f"""
    Eres un experto lingüista. Tu tarea es simplificar esta frase de Lewis Carroll 
    a un lenguaje extremadamente plano, aburrido y moderno, manteniendo el sentido:
    "{frase_original}"
    Respuesta corta, solo la frase simplificada.
    """
    response = ollama.generate(model='llama3.2', prompt=prompt)
    return response['response'].strip()

In [ ]:
# 2. Procesamiento con guardado incremental y recuperación de estado
total_frases = len(texto_alicia)
output_file = "../data/synthetic_alicia_ollama.jsonl"
checkpoint_file = "../data/checkpoint_ollama.txt"

# Leer checkpoint si existe (última posición procesada)
start_index = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as cf:
        start_index = int(cf.read().strip())
    print(f"Reanudando desde la frase {start_index + 1}")

contador = start_index

# Abrir archivo en modo append si continuamos, o write si empezamos de cero
mode = "a" if start_index > 0 else "w"
try:
    with open(output_file, mode, encoding="utf-8") as f:
        for i, frase in enumerate(texto_alicia[start_index:], start=start_index + 1):
            try:
                input_plano = generar_par_sintetico(frase)
                entry = {
                    "input": input_plano,
                    "output": frase
                }
                # Guardar inmediatamente cada entrada
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
                f.flush()  # Forzar escritura al disco
                
                # Actualizar checkpoint
                with open(checkpoint_file, "w") as cf:
                    cf.write(str(i))
                
                contador = i
                print(f"Procesado y guardado {i}/{total_frases} frases")
                    
            except Exception as e:
                print(f"⚠️ Error en frase {i}: {e} - Continuando...")
                continue
    
    # Si llegamos aquí, terminamos exitosamente
    print(f"\n✅ Total procesado y guardado: {contador} frases")
    # Eliminar checkpoint al finalizar
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)
        print("Checkpoint eliminado - proceso completado.")
        
except Exception as e:
    print(f"\n💾 Proceso interrumpido. Última frase procesada: {contador}")
    print(f"Para continuar, ejecuta esta celda de nuevo.")